In [ ]:
%load_ext autoreload
%autoreload 2

import plotly.express as px
import pandas as pd
import numpy as np
from scipy.stats import (
    bootstrap,
    gmean,
)
from IPython.display import display

from duet.process import *
from duet.constants import *
from common import *

df_raw = load_raw()
unique_suites = df_raw[RF.suite].unique()
df_raw.describe()

In [ ]:
df = (
    df_raw.groupby(ARTIFACT_COL)
    .agg({RF.iteration: sum, RF.runid: sum, RF.start: min, RF.end: max})
    .reset_index()
)
df[RF.time] = df[RF.end] - df[RF.start]

px.bar(
    df,
    x=RF.time,
    y=AF.hostname,
    orientation="h",
).show()

df = (
    df_raw.groupby(DF.env)
    .agg({RF.iteration: sum, RF.runid: sum, RF.time: sum})
    .reset_index()
)
df[RF.time].apply(lambda x: pd.Timedelta(seconds=x))

fig = px.bar(
    df,
    x=RF.time,
    y=DF.env,
    orientation="h",
)
fig.show()

In [ ]:
df_runtime = run_time(df_raw)
df_runtime[DF.run_time_per_iteration] = (
    df_runtime[DF.run_time] / df_runtime[DF.iteration_count]
)

df_runtime_average = (
    df_runtime.groupby(BENCHMARK_ENV_COL)[DF.run_time_per_iteration]
    .mean()
    .reset_index()
)

df_pivot = df_runtime.pivot_table(
    index=[RF.suite, RF.benchmark, DF.env],
    columns=[RF.type],
    values=[DF.run_time_per_iteration],
).reset_index()
df_pivot[DF.run_time_speedup] = (
    df_pivot[DF.run_time_per_iteration]["seqn"]
    / df_pivot[DF.run_time_per_iteration]["duet"]
) - 1

for suite in unique_suites:
    # Benchmark run duration
    fig = px.box(
        df_runtime[df_runtime[RF.suite] == suite],
        x=RF.benchmark,
        y=DF.run_time,
        color=DF.env,
        facet_col=RF.type,
        title=f"Benchmark Run Time - {suite}",
    )
    fig.update_xaxes(showticklabels=True)
    fig.show()

    # Benchmark run duration
    fig = px.box(
        df_runtime[df_runtime[RF.suite] == suite],
        x=RF.benchmark,
        y=DF.run_time_per_iteration,
        color=DF.env,
        facet_col=RF.type,
        title=f"Benchmark Run Time per iteration - {suite}",
    )
    fig.update_xaxes(showticklabels=True)
    fig.show()

    fig = px.bar(
        df_pivot[df_pivot[RF.suite] == suite],
        x=RF.benchmark,
        y=DF.run_time_speedup,
        color=DF.env,
        barmode="group",
        title=f"Run time speedup async duet vs. sequential - {suite}",
    )
    fig.show()

In [ ]:
fig = px.box(
    df_pivot, x=DF.env, y=DF.run_time_speedup, color=RF.suite, hover_data=[RF.benchmark]
)
fig.update_layout(
    xaxis_title="Environment",
    yaxis_title="Runtime speedup",
    legend_title="Suite",
)
fig.write_image("figures/runtime_speedup.pdf")
fig.show()

In [ ]:
df = run_time(df_raw)

# Suite run duration
df = (
    df.groupby(by=[RF.suite, RF.type, DF.env])
    .agg(suite_duration=(DF.run_time, sum), iteration_count=("iteration_count", sum))
    .reset_index()
)
fig = px.bar(
    df.sort_values(by=RF.type),
    x="suite_duration",
    y=RF.suite,
    color=RF.type,
    facet_row=DF.env,
    barmode="group",
    orientation="h",
    title="Suite Run Duration",
)
fig.update_layout(height=1000)
fig.show()

# Suite run duration per iteration
df[DF.run_time_per_iteration] = df["suite_duration"] / df["iteration_count"]
fig = px.bar(
    df.sort_values(by=RF.type),
    x=DF.run_time_per_iteration,
    y=RF.suite,
    color=RF.type,
    facet_row=DF.env,
    barmode="group",
    orientation="h",
    title="Suite Run Duration per iteration",
)
# fig.update_layout(height=1000)
fig.show()

# Suite speedup per iteration
df = df.pivot_table(
    index=[RF.suite, DF.env], columns=[RF.type], values=[DF.run_time_per_iteration]
).reset_index()
df[DF.run_time_speedup] = (
    df[DF.run_time_per_iteration]["seqn"] / df[DF.run_time_per_iteration]["duet"]
) - 1

df["text"] = df[DF.run_time_speedup].apply(lambda x: f"{(x * 100):.0f} %")
fig = px.bar(
    df,
    y=RF.suite,
    x=DF.run_time_speedup,
    color=DF.env,
    barmode="group",
    text="text",
)
fig.update_layout(
    xaxis_title="Average runtime speedup per iteration",
    yaxis_title="Suite",
    legend_title="Environment",
)
# fig.write_image(f"figures/runtime_speedup.pdf")
fig.show()